# Assignment 5: Create a model deployment CI/CD pipeline
In this assignment you implement a model deployment pipeline. 

There are many possible patterns how you can deploy ML models into production. These patterns are dependent on your environment, for example, use of multi-account structures, testing and approval workflows, number and type of environment stages. In this lab you implement a simple single-account model deployment. However, it has all main elements of a real-world deployment pipeline, such as an automated CI/CD process, model unit testing, a two-stage deployment with a test and a production stages, and an additional approval before deployment into production.

To provision all required infrastructure and components into your AWS account, you use again a built-in [MLOps Project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates.html).

The following diagram shows the end-to-end architecture of the composed CI/CD pipeline comprising a model building and a model deployment pipelines.

![](../img/cicd-project-e2e.png)

The left part of the diagram, is a **model building pipeline**, which you provisioned and configured in the previous notebook. In this notebook, you are going to provision the right part, a **model deployment pipeline**. These two pipelines are connected via the Model Registry. The model building pipeline registers a new version of the model after a successful execution of the pipeline. To launch the model deployment pipeline, you must approve the model version in the Model Registry. The model deployment pipeline first deploys the approved model version into a staging environment, runs model unit tests, and waits for a manual approval. After you approve the **DeployStaging** stage of the CI/CD pipeline, the pipeline continues and deploys the model into a production environment.

The model deployment project follows Infrastructure as Code (IaC) approach by using [AWS Cloud Formation](https://aws.amazon.com/cloudformation/) templates for model deployment. The project also deploys a source code repository with inference endpoint configuration and model unit test code.

Refer to the notebook [`05-deploy.ipynb`](../05-deploy.ipynb) for code snippets and a general guidance for the exercises in this assignment.

## Import packages

In [ ]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep

## Exercise 1: Create a model deployment project
You can re-use code from [`05-deploy.ipynb`](../05-deploy.ipynb) notebook to programmatically create a project using Python SDK `boto3`.

Alternatively you can provision a new project via Studio UX.

In [ ]:
# Get project details after you created the project
# project_name = <YOUR PROJECT NAME>
# sm.describe_project(ProjectName=project_name)

## Exercise 2: Explore project components
The SageMaker MLOps project provisions an end-to-end CI/CD model deployment pipeline in your AWS account:

![](../img/mlops-model-deploy-2.png)

The main components are:
1. An AWS CodeCommit repository with configuration, testing, and workflow code
2. An AWS CodePipeline deployment pipeline with four stages
3. An Amazon EventBridge rule to launch a CodePipeline pipeline execution when a model package version is approved or rejected
4. A manual approval step after the model unit tests to deploy the production inference endpoint

Explore each of the pipeline components in AWS console.

### AWS CodeCommit repository
[Clone](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html#sagemaker-proejcts-walkthrough-clone) the project code repository to the Studio home directory and explore the source code. Navigate to the project folder on Studio file system.

The `README.md` file provides a detailed overview of the repository structure and the description of each file. The main components of the source code:
- Two build stages each with own `buildspec.yml` file: for retrieving the latest approved model package and building the configuration files, and for unit tests
- Two Python scripts `build.py` and `test/test.py` with executable code for each of the build stages
- A CloudFormation template `endpoint-config-template.yml` for IaC model deployment as real-time inference endpoints
- Two configuration files `staging-config.json` and `prod-config.json`. Pipeline's deployment steps use these files to configure parameters of the inference endpoints, such as compute instance type, data capture configuration, tags

The MLOps project implements only one possible approach for an automated deployment pipeline. You can implement a different approach using a different technology stack depending on your specific environment. You can still use SageMaker Project templates to package any components and technology stack to create a pre-defined CI/CD pipeline templates.

### AWS CodePipeline pipeline
Open [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline/pipelines) and explore the model deployment pipeline named `sagemaker-<project-name>-<project-id>-modeldeploy`. There are four stages in the pipeline. Explore the transitions between stages and artifacts created by each stage.

For example, explore the build stage. Navigate to [AWS CodeBuild console](https://console.aws.amazon.com/codesuite/codebuild/projects) and open a build project with the name `sagemaker-<project-name>-<project-id>-modeldeploy`, open **Environment variables** pane. You can see the environment variables set by the MLOps project template:

![](../img/codebuild-env-variables.png)

Open the `build.py` file in the project folder. Explore the execution flow and how the implementation uses the passed arguments to set configuration parameters and to create the configuration files.

Explore **DeployStaging** stage of the CodePipeline pipeline. Click on the pipeline name in the pipeline list and then click on "i" icon next to **DeployResourceStaging** action of the **DeployStaging** stage. You see how the CloudFormation deploy action is configured:

![](../img/deploystaging-config.png)

Open the `/test/test.py` file in the project folder. Explore how the endpoint tests are done. You can implement `invoke_endpoint` function and send a payload to the endpoint to test it.

Continue exploring other stages of the pipeline, such as manual approval, deployment of the production endpoint.

### Amazon EventBridge rule
Navigate to the rules in the EventBridge [console](https://console.aws.amazon.com/events/home?#/rules) and find rules with the names `sagemaker-<project-name>-<project-id>-code` and `sagemaker-<project-name>-<project-id>-model`. The first rule `...-code` launches the pipeline on each push into the source code repository. The second rule `...-model` launches the pipeline on any model package state change, see the **Event pattern**:

```
{
  "detail-type": ["SageMaker Model Package State Change"],
  "source": ["aws.sagemaker"],
  "detail": {
    "ModelPackageGroupName": ["from-idea-to-prod-model-group"]
  }
}
```

## Exercise 3: Execute the model deployment pipeline
To launch the deployment pipeline you need to approve a model version in the model registry. 

You can approve the model version either in Studio UX in the Model registry or do it programmatically in the notebook.

Follow the pipeline execution in [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline/pipelines). After the pipeline deployed the staging inference endpoint, manually approve the deployment of the production endpoint. See the [`05-deploy.ipynb`](../05-deploy.ipynb) notebook for instructions.

Verify that you have two `InService` endpoints after a successful completion of the deployment pipeline:

<img src="../img/endpoint-prod.png" width="400"/>

## Continue with the assignment 6
Navigate to the [assignment 6](06-assignment-monitoring.ipynb) notebook.